In [12]:
# ML_in_Finance-Interpretability
# Author: Matthew Dixon
# Version: 1.0 (08.09.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6.9 and the following packages:
# numpy=1.18.1, keras=2.3.1, tensorflow=2.0.0, statsmodels=0.10.1, scikit-learn=0.22.1
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F., I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 

# Overview
The purpose of this notebook is to illustrate a neural network interpretability method which is compatible with linear regression. 

In linear regression, provided the independent variables are scaled, one can view the regression coefficients as a measure of importance of the variables. Equivalently, the dependent variable can be differentiated w.r.t. the inputs to give the coefficient. 

Similarly, the derivatives of the network w.r.t. the inputs are a non-linear generalization of interpretability in linear regression. Moreover, we should expect the neural network gradients to approximate the linear regression coefficients when the data is generated by a linear regression model. 

Various simple experimental tests, corresponding to Section 3 of Chpt 5, are performed to illustrate the properties of network interpretability.

In [13]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor
import statsmodels.api as sm
import sklearn

## Simple Data Generation Process (DGP)


Let us generate data from the following linear regression model

$Y=X_1+X_2 + \epsilon~, ~~X_1, X_2 \sim N(0,1)~, ~~\epsilon \sim N(0,1)$

In [14]:
M = 5000 # Number of samples
np.random.seed(7) # Set NumPy's random seed for reproducibility
X = np.zeros(shape=(M, 2))
X[:int(M/2), 0] = np.random.randn(int(M/2))
X[:int(M/2), 1] = np.random.randn(int(M/2))

# Use antithetic sampling to reduce the bias in the mean
X[int(M/2):, 0] = -X[:int(M/2), 0]
X[int(M/2):, 1] = -X[:int(M/2), 1]

eps = np.zeros(shape=(M,1))
eps[:int(M/2)] = np.random.randn(int(M/2), 1)
eps[int(M/2):] = -eps[:int(M/2)]
Y = X[:, 0] + X[:, 1] + eps.flatten()

## Use ordinary least squares to fit a linear model to the data
For a baseline, let us compare the neural network with OLS regression. 

We fit statsmodels' OLS model to the data

In [15]:
ols_results = sm.OLS(Y, sm.add_constant(X)).fit()

For each input, get the predicted $Y$ value according to the model

In [16]:
y_ols = ols_results.predict(sm.add_constant(X))

View characteristics of the resulting model. You should observe that the intercept is close to zero and the other coefficients are close to one.

In [17]:
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.678
Model:                            OLS   Adj. R-squared:                  0.677
Method:                 Least Squares   F-statistic:                     5249.
Date:                Mon, 21 Sep 2020   Prob (F-statistic):               0.00
Time:                        23:03:33   Log-Likelihood:                -7020.4
No. Observations:                5000   AIC:                         1.405e+04
Df Residuals:                    4997   BIC:                         1.407e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.939e-18      0.014  -4.98e-16      1.000      -0.027       0.027
x1             0.9858      0.014     70.409      0.000       0.958       1.013
x2             1.0190      0.014     72.695      0.000       0.992       1.047
==============================================================================
Omnibus:                        0.800   Durbin-Watson:                   1.941
Prob(Omnibus):                  0.670   Jarque-Bera (JB):                0.750
Skew:                          -0.000   Prob(JB):                        0.687
Kurtosis:                       3.060   Cond. No.                         1.02
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Compare with a feedforward NN with no hidden layers

Recall that the feedforward network with no hidden layers or activation function is a linear regression model.

Create a build function for the linear perceptron, which transforms the inputs directly to a single output

In [18]:
def linear_NN0_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(1, input_dim=2, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

An early stopping callback to terminate training once the weights appear to have converged to an optimum. 

In [19]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

Passing the build function for our model and training parameters to the `KerasRegressor` constructor to create a Scikit-learn-compatible regression model. This allows you to take advantage of the library's built-in tools and estimator methods, and to incorporate it into Scikit-learn pipelines. 

In [20]:
lm = KerasRegressor(build_fn=linear_NN0_model, epochs=40, batch_size=10, verbose=1, callbacks=[es])

Train the model

In [21]:
lm.fit(X, Y)

Epoch 1/40
500/500 [==============================] - 0s 752us/step - loss: 2.2519 - mae: 1.2013 - mse: 2.2519
Epoch 2/40
500/500 [==============================] - 0s 737us/step - loss: 1.4090 - mae: 0.9537 - mse: 1.4090
Epoch 3/40
500/500 [==============================] - 0s 738us/step - loss: 1.0856 - mae: 0.8327 - mse: 1.0856
Epoch 4/40
500/500 [==============================] - 0s 779us/step - loss: 0.9926 - mae: 0.7957 - mse: 0.9926
Epoch 5/40
500/500 [==============================] - 0s 774us/step - loss: 0.9741 - mae: 0.7877 - mse: 0.9741
Epoch 6/40
500/500 [==============================] - 0s 737us/step - loss: 0.9717 - mae: 0.7869 - mse: 0.9717
Epoch 7/40
500/500 [==============================] - 0s 791us/step - loss: 0.9712 - mae: 0.7866 - mse: 0.9712
Epoch 8/40
500/500 [==============================] - 0s 751us/step - loss: 0.9715 - mae: 0.7868 - mse: 0.9715
Epoch 9/40
500/500 [==============================] - 0s 822us/step - loss: 0.9713 - mae: 0.7866 - mse: 0.9713
E

### Check that the weights are close to one
The weights should be close to unity. The bias term is the second entry and should be close to zero.

In [22]:
print("weights: " + str(lm.model.layers[0].get_weights()[0]))
print("bias: " + str(lm.model.layers[0].get_weights()[1]))

weights: [[0.98217595]
 [1.0337954 ]]
bias: [-0.00366889]


## Compare with a FFW Neural Network with one hidden layer (unactivated)


This time we create a neural network with a hidden layer with 10 units.

In [23]:
n = 10 # number of hidden units

In [24]:
def linear_NN1_model(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal')) 
    model.add(Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [25]:
lm = KerasRegressor(build_fn=linear_NN1_model, epochs=50, batch_size=10, verbose=1, callbacks=[es])

In [26]:
lm.fit(X, Y)

Epoch 1/50
500/500 [==============================] - 0s 821us/step - loss: 1.5718 - mae: 0.9785 - mse: 1.5718
Epoch 2/50
500/500 [==============================] - 0s 856us/step - loss: 0.9733 - mae: 0.7873 - mse: 0.9733
Epoch 3/50
500/500 [==============================] - 0s 831us/step - loss: 0.9731 - mae: 0.7873 - mse: 0.9731
Epoch 4/50
500/500 [==============================] - 0s 840us/step - loss: 0.9738 - mae: 0.7876 - mse: 0.9738
Epoch 5/50
500/500 [==============================] - 0s 827us/step - loss: 0.9729 - mae: 0.7871 - mse: 0.9729
Epoch 6/50
500/500 [==============================] - 0s 837us/step - loss: 0.9729 - mae: 0.7870 - mse: 0.9729
Epoch 7/50
500/500 [==============================] - 0s 831us/step - loss: 0.9732 - mae: 0.7874 - mse: 0.9732
Epoch 8/50
500/500 [==============================] - 0s 846us/step - loss: 0.9730 - mae: 0.7876 - mse: 0.9730
Epoch 9/50
500/500 [==============================] - 0s 884us/step - loss: 0.9738 - mae: 0.7887 - mse: 0.9738
E

In [27]:
W1 = lm.model.get_weights()[0]
b1 = lm.model.get_weights()[1]
W2 = lm.model.get_weights()[2]
b2 = lm.model.get_weights()[3]
print(W1, W2)

[[-0.3773278  -0.2236989  -0.35481596 -0.34812185  0.3253199   0.26338968
   0.30855018  0.38172016  0.39396024  0.22428878]
 [-0.26117954 -0.36914834 -0.35925072 -0.30433056  0.34873417  0.34487224
   0.25501597  0.40840706  0.3272932   0.34817845]] [[-0.28649876]
 [-0.33347732]
 [-0.2902729 ]
 [-0.32563964]
 [ 0.30829367]
 [ 0.3128368 ]
 [ 0.32633594]
 [ 0.27538082]
 [ 0.30303347]
 [ 0.31902447]]


### Check that the coefficients are close to one and the intercept is close to zero

In [28]:
beta_0 = np.dot(np.transpose(W2), b1) + b2
beta_1 = np.dot(np.transpose(W2), W1[0])
beta_2 = np.dot(np.transpose(W2), W1[1])

In [29]:
print(beta_0, beta_1, beta_2)

[0.00052501] [0.9784964] [1.0226609]


## Compare with a feedforward NN with one hidden layer ($tanh$ activated)

Finally, we create another model with a 10 unit hidden layer, this time with a $tanh$ activation function.

In [30]:
# number of hidden neurons
n = 10

In [31]:
# with non-linear activation
def linear_NN1_model_act(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model

In [32]:
lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=1, callbacks=[es])

Train the model

In [33]:
lm.fit(X, Y)

Epoch 1/100
500/500 [==============================] - 0s 822us/step - loss: 1.6636 - mae: 1.0117 - mse: 1.6636
Epoch 2/100
500/500 [==============================] - 0s 830us/step - loss: 0.9921 - mae: 0.7957 - mse: 0.9921
Epoch 3/100
500/500 [==============================] - 0s 815us/step - loss: 0.9905 - mae: 0.7954 - mse: 0.9905
Epoch 4/100
500/500 [==============================] - 0s 849us/step - loss: 0.9882 - mae: 0.7946 - mse: 0.9882
Epoch 5/100
500/500 [==============================] - 0s 839us/step - loss: 0.9870 - mae: 0.7937 - mse: 0.9870
Epoch 6/100
500/500 [==============================] - 0s 866us/step - loss: 0.9854 - mae: 0.7929 - mse: 0.9854
Epoch 7/100
500/500 [==============================] - 0s 814us/step - loss: 0.9834 - mae: 0.7921 - mse: 0.9834
Epoch 8/100
500/500 [==============================] - 0s 813us/step - loss: 0.9830 - mae: 0.7927 - mse: 0.9830
Epoch 9/100
500/500 [==============================] - 0s 851us/step - loss: 0.9822 - mae: 0.7920 - mse:

### Compute the Sensitivities

In [34]:
# Assumes that the activation function is tanh
def sensitivities(lm, X):
    
    W1 = lm.model.get_weights()[0]
    b1 = lm.model.get_weights()[1]
    W2 = lm.model.get_weights()[2]
    b2 = lm.model.get_weights()[3]
    
    
    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta = np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)
    
    beta[:, 0] = (np.dot(np.transpose(W2), np.tanh(b1)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):
 
        Z1 = np.tanh(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1)
      
        D = np.diag(1 - Z1**2)
        
        for j in range(p):  
            beta[i, j+1] = np.dot(np.transpose(W2), np.dot(D, W1[j]))
            
    return beta

In [35]:
beta = sensitivities(lm, X)

### Check that the intercept is close to one and the coefficients are close to one

In [36]:
print(np.mean(beta, axis=0))

[0.02807781 0.9681357  1.0832785 ]


In [37]:
print(np.std(beta, axis=0))

[1.4044788e-06 6.2763922e-02 7.0253789e-02]


In [38]:
np.histogram()

TypeError: ignored